# Welcome to the Lab 🥼🧪

### Supply-Demand Gap & Price Cut Analysis

In this notebook, we'll demonstrate how to analyze market dynamics using two key indicators:
1. The relationship between housing supply and demand
2. Price reduction trends across markets

**You'll learn how to:**
* Search for specific markets and analyze their dynamics
* Compare local trends against national benchmarks
* Create publication-ready visualizations of market conditions
* Export the underlying data for further analysis

**Need help getting started?**
* Get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along
* To run this immediately, you can use Google Colab. Remember, you must set your PARCL_LABS_API_KEY.

Run in --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/market_analysis/gap_and_price_cut_analysis.ipynb)

In [ ]:
# --- SETUP CODE BLOCK ---

# First, install and/or upgrade to the latest version of required packages
%pip install --upgrade parcllabs nbformat geopandas

# Import required libraries
import os
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import timedelta
import plotly.graph_objects as go
from parcllabs import ParclLabsClient

# Additional imports for styling
from parcllabs.beta.charting.styling import default_style_config as style_config
from parcllabs.beta.charting.utils import create_labs_logo_dict, save_figure
from parcllabs.beta.ts_stats import TimeSeriesAnalysis

# Input your API key here
# If you don't have an API key, sign up at https://dashboard.parcllabs.com/signup
PARCL_LABS_API_KEY = "" # Replace with your API key

# Initialize the Parcl Labs Client
try:
    client = ParclLabsClient(
        api_key=PARCL_LABS_API_KEY or os.environ.get('PARCL_LABS_API_KEY'),
        limit=1000,
        turbo_mode=True
    )

    # Test the client with a simple API call
    test_market = client.search.markets.retrieve(query='United States', limit=1)
    print("✓ API connection successful!")

except Exception as e:
    print("\n❌ Error initializing the Parcl Labs client!")
    print("\nTo fix this:")
    print("1. Sign up for an API key at https://dashboard.parcllabs.com/signup")
    print("2. Either:")
    print("   a) Replace the empty string in PARCL_LABS_API_KEY = \"\" with your key")
    print("   or")
    print("   b) Set your PARCL_LABS_API_KEY environment variable")
    print(f"\nError details: {str(e)}")
    raise

# Define date ranges and constants
START_DATE = '2022-09-01'  # Data series begins September 1, 2022
END_DATE = '2024-12-09'    # Adjust this date as needed
USA_MARKET_ID = 5826765    # This is the Parcl ID for the USA market (used for benchmarking)

print("✓ Setup complete! You're ready to start analyzing markets.")

✓ API connection successful!
✓ Setup complete! You're ready to start analyzing markets.


# Market Selection 🔍

Before analyzing supply-demand gaps and price cuts, you'll need to select a market. You can:
* Search by market name (e.g., 'Atlanta', 'Miami', 'Los Angeles')
* Specify location type (optional) to narrow results (e.g., 'CBSA', 'CITY', 'ZIP5')
* Choose from multiple results if your search returns several options

The tool will also automatically set up USA-level data for national benchmarking.

In [ ]:
# --- MARKET SEARCH CODE BLOCK ---

# Prompt the user for the market name
print("🔍 Let's find your market!")
market_query = input("Enter the market name (e.g., 'Atlanta', 'New York', 'Miami'): ").strip()

# Optionally specify location type
print("\nOptional: Specify a location type to narrow your search")
print("Common types: 'CBSA' (metro area), 'CITY', 'ZIP5', 'COUNTY'")
print("Press Enter to search all types")
location_type_input = input("Enter location type or press Enter to skip: ").strip()
if location_type_input == '':
    location_type_input = None

# Prepare search arguments
search_kwargs = {'query': market_query}
if location_type_input is not None:
    search_kwargs['location_type'] = location_type_input

try:
    # Perform the search
    market_df = client.search.markets.retrieve(**search_kwargs, limit=5)

    if market_df.empty:
        raise ValueError("No markets found for this query. Please try another search.")

    # If multiple results are returned, let the user pick one
    if len(market_df) > 1:
        print("\n📍 Multiple markets found:")
        for i, row in market_df.iterrows():
            print(f"{i}: {row['name']} (Type: {row['location_type']}, ID: {row['parcl_id']})")

        selection = input("\nEnter the number of the market you want to analyze: ").strip()
        try:
            selection_idx = int(selection)
            if selection_idx < 0 or selection_idx >= len(market_df):
                raise ValueError
        except ValueError:
            raise ValueError("Invalid selection. Please restart and choose a valid number.")
        selected_market = market_df.iloc[selection_idx]
    else:
        selected_market = market_df.iloc[0]

    # Store selected market information
    selected_market_id = selected_market['parcl_id']
    selected_market_name = selected_market['name']
    print(f"\n✓ Selected Market: {selected_market_name} (ID: {selected_market_id})")

    # Retrieve the US market for benchmarking
    us = client.search.markets.retrieve(query='United States', limit=1)
    print("\n✓ USA benchmark data ready")
    print("\nMarket selection complete! Ready to analyze market dynamics.")

except Exception as e:
    print(f"\n❌ Error during market search: {str(e)}")
    print("Please try again with a different market name or location type.")
    raise

🔍 Let's find your market!


# Data Retrieval 📊

Now that we've selected a market, we'll gather three key datasets:

1. **Supply Data**


2. **Demand Data**


3. **Price Changes**

These metrics together will give us a comprehensive view of market dynamics.

In [ ]:
# --- DATA RETRIEVAL CODE BLOCK ---

try:
    print("📥 Retrieving market data...")

    # 1. Retrieve weekly supply (inventory) data
    print("\nFetching supply data...", end='')
    supply_df = client.for_sale_market_metrics.for_sale_inventory.retrieve(
        parcl_ids=[selected_market_id, USA_MARKET_ID],
        start_date=START_DATE,
        end_date=END_DATE,
        auto_paginate=True
    )
    print(" ✓")

    # 2. Retrieve monthly demand (sales) data
    print("Fetching demand data...", end='')
    demand_df = client.market_metrics.housing_event_counts.retrieve(
        parcl_ids=[selected_market_id, USA_MARKET_ID],
        start_date=START_DATE,
        end_date=END_DATE,
        auto_paginate=True
    )
    print(" ✓")

    # 3. Retrieve weekly price reduction data
    print("Fetching price change data...", end='')
    price_changes_df = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
        parcl_ids=[selected_market_id, USA_MARKET_ID],
        start_date=START_DATE,
        end_date=END_DATE,
        auto_paginate=True
    )
    print(" ✓")

    # Summarize the data retrieved
    print("\n✓ Data retrieval complete!")
    print(f"\nDataset Summary:")
    print(f"• Supply Data: {len(supply_df)} weekly records")
    print(f"• Demand Data: {len(demand_df)} monthly records")
    print(f"• Price Changes: {len(price_changes_df)} weekly records")

    # Validate data quality
    if len(supply_df) == 0 or len(demand_df) == 0 or len(price_changes_df) == 0:
        print("\n⚠️ Warning: One or more datasets are empty. This might affect the analysis.")

except Exception as e:
    print("\n❌ Error retrieving market data!")
    print(f"Error details: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Check your internet connection")
    print("2. Verify your API key has not expired")
    print("3. Confirm the selected market_id is valid")
    raise

📥 Retrieving market data...

Fetching supply data...data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01', 'end_date': '2024-12-09'}, params: {}
 ✓
Fetching demand data...data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01', 'end_date': '2024-12-09'}, params: {}
 ✓
Fetching price change data...data: {'parcl_id': ['2899750', '5826765'], 'start_date': '2022-09-01', 'end_date': '2024-12-09'}, params: {}
 ✓

✓ Data retrieval complete!

Dataset Summary:
• Supply Data: 238 weekly records
• Demand Data: 54 monthly records
• Price Changes: 238 weekly records


In [ ]:
# --- DATA PREPARATION CODE BLOCK ---

print("\n📊 Preparing data for analysis...")

try:
    # Convert weekly data to monthly and compute rolling statistics
    supply_monthly = (
        supply_df
        .merge(price_changes_df[['parcl_id', 'date', 'count_price_drop']], on=['parcl_id','date'])
        .assign(
            pct_price_drops=lambda df: df['count_price_drop'] / df['for_sale_inventory'],
            date=lambda df: df['date'].dt.to_period('M').dt.to_timestamp()
        )
        .groupby(['parcl_id','date'], as_index=False)
        .agg({'for_sale_inventory':'median','pct_price_drops':'median'})
        .sort_values(['parcl_id','date'])
        .assign(ma_pct_price_drops=lambda df: df.groupby('parcl_id')['pct_price_drops']
               .transform(lambda x: x.rolling(window=3).mean()))
    )

    # Combine supply and demand data for analysis
    supply_demand_data = (
        demand_df
        .assign(date=lambda df: df['date'].dt.to_period('M').dt.to_timestamp())
        .merge(supply_monthly[['parcl_id','date','for_sale_inventory']],
               on=['parcl_id','date'], how='inner')
        .sort_values(['parcl_id','date'])
        .assign(
            pct_change_demand=lambda df: df.groupby('parcl_id')['sales'].pct_change(periods=12),
            pct_change_supply=lambda df: df.groupby('parcl_id')['for_sale_inventory'].pct_change(periods=12),
            ma_pct_change_demand=lambda df: df.groupby('parcl_id')['pct_change_demand']
                .transform(lambda x: x.rolling(window=3).mean()),
            ma_pct_change_supply=lambda df: df.groupby('parcl_id')['pct_change_supply']
                .transform(lambda x: x.rolling(window=3).mean())
        )
    )

    print("✓ Data preparation complete!")
    print("\nCalculated metrics include:")
    print("• Year-over-year supply and demand changes")
    print("• 3-month moving averages")
    print("• Price reduction percentages")

except Exception as e:
    print("\n❌ Error during data preparation!")
    print(f"Error details: {str(e)}")
    print("\nPlease ensure all required data was retrieved successfully.")
    raise


📊 Preparing data for analysis...
✓ Data preparation complete!

Calculated metrics include:
• Year-over-year supply and demand changes
• 3-month moving averages
• Price reduction percentages


# Visualizations 📈

## Supply-Demand Gap Analysis

First, we'll create a bar chart comparing year-over-year changes in supply (inventory) and demand (sales). This visualization helps identify:
* Market imbalances between supply and demand
* Recent trends (last 6 months)
* Potential market opportunities or challenges

The chart shows:
* 🔵 Blue bars: Year-over-year change in demand (sales)
* 🟡 Yellow bars: Year-over-year change in inventory (supply)

In [ ]:
# --- SUPPLY-DEMAND BAR CHART CODE BLOCK ---

print("📊 Creating supply-demand gap visualization...")

try:
    # Filter data for visualization
    market_sd = (
        supply_demand_data.query("parcl_id == @selected_market_id")
        .dropna(subset=['ma_pct_change_demand','ma_pct_change_supply'])
    )

    # Get the last 6 months of data
    end_date = market_sd['date'].max()
    if pd.isnull(end_date):
        print("\n⚠️ No data available for visualization")
        filtered_sd = market_sd
    else:
        six_months_ago = end_date - pd.DateOffset(months=6)
        filtered_sd = market_sd.query("date >= @six_months_ago")
        print(f"\nAnalyzing data from {six_months_ago.strftime('%B %Y')} to {end_date.strftime('%B %Y')}")

    # Prepare data for visualization
    market_for_bar = pd.melt(
        filtered_sd,
        id_vars=['date'],
        value_vars=['ma_pct_change_demand','ma_pct_change_supply'],
        var_name='type',
        value_name='percent_change'
    ).replace({
        'ma_pct_change_demand':'Demand (Sales)',
        'ma_pct_change_supply':'Supply (Inventory)'
    })

    # Create the bar chart
    fig_bar = px.bar(
        market_for_bar,
        x='date',
        y='percent_change',
        color='type',
        barmode='group',
        title=f"YoY Change in Supply and Demand in {selected_market_name} (Last 6 Months)",
        labels={'percent_change':'Percent Change','date':''},
        color_discrete_map={'Demand (Sales)':'#448CF2','Supply (Inventory)':'#FCC054'}
    )

    # Apply Parcl Labs styling
    fig_bar.update_layout(
        margin=dict(l=60, r=100, t=100, b=60),
        title={
            'y':0.98,
            'x':0.5,
            'xanchor':'center',
            'yanchor':'top',
            'font':dict(family="Helvetica Neue", size=24, color='#FFFFFF')
        },
        xaxis=dict(
            showgrid=True,
            gridwidth=0.7,
            griddash='dash',
            gridcolor='#132D59',
            linecolor='#132D59',
            linewidth=2,
            titlefont=dict(size=18, color='#FFFFFF'),
            tickfont=dict(size=14, color='#FFFFFF'),
            tickangle=0
        ),
        yaxis=dict(
            title='Percent Change',
            showgrid=True,
            gridwidth=0.7,
            griddash='dash',
            gridcolor='#132D59',
            linecolor='#132D59',
            zeroline=False,
            tickformat='.0%',
            linewidth=2,
            titlefont=dict(size=18, color='#FFFFFF'),
            tickfont=dict(size=14, color='#FFFFFF')
        ),
        plot_bgcolor='#080D16',
        paper_bgcolor='#080D16',
        font=dict(family="Helvetica Neue", color='#FFFFFF'),
        legend_title_text='',
        width=1000,
        height=600,
        legend=dict(
            font=dict(size=14, color='#FFFFFF'),
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )

    # Add Parcl Labs logo
    fig_bar.add_layout_image(create_labs_logo_dict())

    # Display the chart
    fig_bar.show()

    # Save the visualization
    save_figure(fig_bar,
                save_path=f"supply_demand_bar_chart_{selected_market_name}.png",
                width=1000,
                height=600)

    print("\n✓ Supply-Demand bar chart created and saved!")
    print(f"📄 Saved as: supply_demand_bar_chart_{selected_market_name}.png")

except Exception as e:
    print("\n❌ Error creating supply-demand visualization!")
    print(f"Error details: {str(e)}")
    print("\nPlease ensure all data preparation steps completed successfully.")
    raise

# Price Reduction Analysis 📉

Next, we'll analyze how frequently sellers are reducing their asking prices - a key indicator of market dynamics. This visualization shows:

* Percentage of inventory with price reductions over time
* Comparison between your selected market and the national average
* 🔵 Blue line: Your selected market's price reduction trend
* ⚪ Gray line: USA benchmark

A rising trend in price reductions often indicates:
* Sellers adjusting to market conditions
* Potential opportunities for buyers
* Shifting market sentiment

In [ ]:
# --- PRICE CUTS LINE CHART CODE BLOCK ---

print("📈 Creating price reduction trend visualization...")

try:
    # Prepare data for the line chart
    line_chart_data = (
        supply_monthly
        .query("parcl_id in [@selected_market_id, @USA_MARKET_ID]")
        .merge(
            pd.DataFrame({
                'parcl_id': [selected_market_id, USA_MARKET_ID],
                'clean_name': [selected_market_name, 'USA']
            }),
            on='parcl_id'
        )
        .dropna(subset=['ma_pct_price_drops'])
        .sort_values('date')
    )

    # Get latest metrics for context
    latest_date = line_chart_data.loc[line_chart_data['clean_name'] == selected_market_name, 'date'].max()
    latest_value = line_chart_data[
        (line_chart_data['clean_name'] == selected_market_name) &
        (line_chart_data['date'] == latest_date)
    ]['ma_pct_price_drops'].values[0]

    print(f"\nLatest data point: {latest_date.strftime('%B %Y')}")
    print(f"Current price reduction rate: {latest_value:.1%}")

    # Create the line chart
    fig_line = px.line(
        line_chart_data,
        x='date',
        y='ma_pct_price_drops',
        color='clean_name',
        labels={'ma_pct_price_drops': '% Price Reductions'},
        title=f"Inventory with Price Reductions<br>{selected_market_name} vs. USA",
        color_discrete_map={selected_market_name: '#448CF2', 'USA': '#334D73'}
    )

    # Enhance line styles for clarity
    for trace in fig_line.data:
        if trace.name == selected_market_name:
            # Make selected market line stand out
            trace.update(line=dict(color='#448CF2', width=4))
        else:
            trace.update(line=dict(color='#334D73', width=3))

    # Add highlight marker for latest data point
    fig_line.add_trace(
        go.Scatter(
            x=[latest_date],
            y=[latest_value],
            mode='markers',
            marker=dict(size=10, color='#448CF2', line=dict(width=2, color='white')),
            showlegend=False
        )
    )

    # Set x-axis range with padding
    x_max = latest_date + pd.Timedelta(days=30)
    x_min = line_chart_data['date'].min()

    # Apply Parcl Labs styling
    fig_line.update_layout(
        margin=dict(l=70, r=80, t=120, b=70),
        title={
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(family="Helvetica Neue", size=24, color='#FFFFFF')
        },
        xaxis=dict(
            title="",
            range=[x_min, x_max],
            showgrid=True,
            gridwidth=0.7,
            griddash="dash",
            gridcolor="#132D59",
            linecolor="#132D59",
            linewidth=2,
            titlefont=dict(color="#FFFFFF"),
            tickfont=dict(size=14, color="#FFFFFF")
        ),
        yaxis=dict(
            title="% Price Reductions",
            showgrid=True,
            gridwidth=0.7,
            griddash="dash",
            gridcolor="#132D59",
            tickfont=dict(size=14, color="#FFFFFF"),
            zeroline=False,
            tickformat=".0%",
            linecolor="#132D59",
            linewidth=2,
            titlefont=dict(size=18, color="#FFFFFF")
        ),
        plot_bgcolor="#080D16",
        paper_bgcolor="#080D16",
        font=dict(family="Helvetica Neue", color="#FFFFFF"),
        showlegend=True,
        width=1000,
        height=600,
        legend=dict(
            font=dict(size=14, color='#FFFFFF'),
            bgcolor='rgba(0,0,0,0)',
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )

    # Add Parcl Labs logo
    fig_line.add_layout_image(create_labs_logo_dict())

    # Display the chart
    fig_line.show()

    # Save the visualization
    save_figure(fig_line,
                save_path=f"price_cuts_line_chart_{selected_market_name}.png",
                width=1000,
                height=600)

    print("\n✓ Price cuts line chart created and saved!")
    print(f"📄 Saved as: price_cuts_line_chart_{selected_market_name}.png")

except Exception as e:
    print("\n❌ Error creating price cuts visualization!")
    print(f"Error details: {str(e)}")
    print("\nPlease ensure all data preparation steps completed successfully.")
    raise

📈 Creating price reduction trend visualization...

Latest data point: December 2024
Current price reduction rate: 51.7%



✓ Price cuts line chart created and saved!
📄 Saved as: price_cuts_line_chart_Denver-Aurora-Lakewood, Co.png


# Export Data 💾

Finally, you can export the underlying data for further analysis. The export will include:

* Supply-demand metrics
* Price reduction trends
* Market comparison data

You can use this data to:
* Perform custom analyses
* Create your own visualizations
* Import into other tools
* Share with your team

In [ ]:
# --- DATA EXPORT CODE BLOCK ---

print("\nWould you like to download the data as CSV files? (yes/no): ")
download_choice = input().strip().lower()

if download_choice in ['yes', 'y']:
    try:
        # Export the data files
        line_chart_data.to_csv(f"price_cuts_data_{selected_market_name}.csv", index=False)
        market_for_bar.to_csv(f"supply_demand_data_{selected_market_name}.csv", index=False)

        print("\n✓ Data exported successfully as:")
        print(f"• price_cuts_data_{selected_market_name}.csv")
        print(f"• supply_demand_data_{selected_market_name}.csv")

    except Exception as e:
        print(f"\n❌ Error exporting data: {str(e)}")
        print("Please check your write permissions and try again.")
else:
    print("\nNo CSV export requested.")